In [5]:
import os

In [6]:
%pwd

'c:\\Users\\k.hegde\\Desktop\\learnings\\E2E_MLflow\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\k.hegde\\Desktop\\learnings\\E2E_MLflow'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
pip install zipfile

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [16]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [17]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(
                f"File: {filename} downloaded with following info: \n{headers}"
            )
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File extracted to {unzip_path}")

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-30 16:21:23,827: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-30 16:21:23,829: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-30 16:21:23,831: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-30 16:21:23,832: INFO: common: created directory at: artifacts]
[2025-10-30 16:21:23,833: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-30 16:21:23,834: INFO: 460543209: File already exists of size: ~ 23 KB]
[2025-10-30 16:21:23,838: INFO: 460543209: File extracted to artifacts/data_ingestion]
